# Vanilla option discrete pricing 
## European calls and puts
The function below takes user inputs for all the necessary variables to price European call and put options using a constant-scale binomial model (constrained to UD = 1). The pricing is achieved by multiplying the end contingent claims by their corresponding risk-neutral probabilities and then discounting by the risk-free rate. This method is equivalent to "back-stepping" through the tree and constructing a self-financing, replicating, previsible trading strategy, as the risk-neutral probability equation is defined from these replicating portfolios. The chosen approach, however, is much less computationally expensive.

In [2]:
import numpy as np
from scipy.special import comb

def option_price(S_0, K, U, N, r = 0, T = 1, option_type = "C"):
    """
    Prices a variety of European call and put options using a binomial model
    
    Parameters:
    S_0: Initial asset price
    K: Strike price
    U: The scale of the asset's "up" movement
    N: Number of time steps
    r: Interest rate
    T: Number of years
    option_type: Type of option (either call : "C" or put : "P" (although any other input will work for put))
    """
    
    dt = T/N # Divide the options life-span equally by the number of steps taken
    D = 1/U # Using UD = 1
    q_h = (np.exp(r*dt) - D)/(U-D) # risk-neutral probability of "up" movement
    q_l = 1 - q_h # risk-neutral probability of "down" movement
    
    # array for number of "up" moves, converted to float so we can use numpy's "power" method 
    no_of_up_moves = np.arange(N, -1, -1, dtype=float).reshape(-1, 1) 
    
    # array of asset prices at the end of our time period 
    final_node_values = S_0 * np.power(U, 2 * no_of_up_moves - N)

    # risk-neutral measure 
    rn_probs = comb(N, no_of_up_moves)  * np.power(q_h, no_of_up_moves) * np.power(q_l, N - no_of_up_moves)
    
    # Calculate the payoff at each final node, depending on whether a call or put was selected
    if option_type == "C":
        final_node_payoff = np.maximum(0, final_node_values - K)
    else:
        final_node_payoff = np.maximum(0, K - final_node_values)
    
    # Expected payoff w.r.t the risk-neutral measure discounted at the continuously compounded rate
    return f"The corresponding option is valued at £{round((np.exp(-r*T) * sum(rn_probs * final_node_payoff))[0], 2)}"

Here, I calculated the price of a call option with a strike price of £120, based on an underlying asset currently priced at £100, expiring in 3 months, with a risk-free rate of 5%. I used 100 steps and assumed the "up" scaling at each step is 1.5. As you can see, the theoretical price is approximately £95.31.

In [52]:
option_price(100, 120, 1.5, 100, r = 0.05, T = 0.25, option_type = "C")

'The corresponding option is valued at £95.31'

I will now value European calls and puts using the Cox-Ross-Rubinstein model. This model selects values for U and D as follows:

$$
\begin{array}{cc} 
U = e^{\sigma*\sqrt{dt}} & D = e^{-\sigma*\sqrt{dt}} \\ 
\end{array}
$$

Using these values results in the convergence of our binomial option pricing model to the Black Scholes model, as $N \to \infty$. Additionally, I've incorporated the option for the user to include a continuous dividend yield from the asset.

In [4]:
def CRR_option_price(S_0, K, sigma, q, N = 1000, r = 0, T = 1, option_type = "C"):
    """
    Prices a variety of European call and put options using a Cox-Ross-Rubinstein valuation
    
    Parameters:
    S_0: Initial asset price
    K: Strike price
    sigma: Asset volatility
    q: Continuous dividend yield
    N: Number of time steps
    r: Interest rate
    T: Number of years
    option_type: Type of option (either call : "C" or put : "P" (although any other input will work for put))
    """
    
    dt = T/N # Divide the options life-span equally by the number of steps taken
    U = np.exp(sigma*np.sqrt(dt)) # CRR equation 
    D = 1/U # Using UD = 1
    q_h = (np.exp((r-q)*dt) - D)/(U-D) # risk-neutral probability of "up" movement
    q_l = 1 - q_h # risk-neutral probability of "down" movement
    
    # array for number of "up" moves, converted to float so we can use numpy's "power" method 
    no_of_up_moves = np.arange(N, -1, -1, dtype=float).reshape(-1, 1) 
    
    # array of asset prices at the end of our time period 
    final_node_values = S_0 * np.power(U, 2 * no_of_up_moves - N)

    # risk-neutral measure 
    rn_probs = comb(N, no_of_up_moves)  * np.power(q_h, no_of_up_moves) * np.power(q_l, N - no_of_up_moves)
    
    # Calculate the payoff at each final node, depending on whether a call or put was selected
    if option_type == "C":
        final_node_payoff = np.maximum(0, final_node_values - K)
    else:
        final_node_payoff = np.maximum(0, K - final_node_values)
    
    # Expected payoff w.r.t the risk-neutral measure discounted at the continuously compounded rate
    return f"The corresponding option is valued at £{round((np.exp(-r*T) * sum(rn_probs * final_node_payoff))[0], 2)}"

[**NOTE**]: I have intentionally set a high default number of steps in the tree to approach as closely as possible to the theoretical price from the Black-Scholes model.

Here, I calculated the price of a put option with a strike price of £60, based on an underlying asset currently priced at £50, with a volatility of 20%, expiring in 6 months. The risk-free rate is 8%, and there is a continuous dividend yield of 4%. As you can see, the theoretical price is approximately £9.11.

In [53]:
CRR_option_price(50, 60, 0.2, 0.04, r = 0.08, T=0.5, option_type = "P")

'The corresponding option is valued at £9.11'

## American calls and puts

Here, I've created a function to price American options on assets that earn a continuous dividend yield, again using the Cox-Ross-Rubinstein model. Since, by definition, American options can be exercised at any point during their maturity, we cannot price them the same way we priced European options earlier in this notebook (i.e., by only considering the final node values). Instead, we have to "back-step" through the tree, calculating the option price at each parent node by taking the discounted expectation of the claim at the child nodes with respect to the risk-neutral measure. We then compare this with the option payoff from exercising at that parent node. The maximum value of the two will be used when travelling further down the tree.

In [10]:
def american_option_price(S_0, K, sigma, q, N = 1000, r = 0, T = 1, option_type = "C"):
    """
    Prices a variety of American call and put options using a Cox-Ross-Rubinstein valuation
    
    Parameters:
    S_0: Initial asset price
    K: Strike price
    sigma: Asset volatility
    q: Continuous dividend yield
    N: Number of time steps
    r: Interest rate
    T: Number of years
    option_type: Type of option (either call : "C" or put : "P" (although any other input will work for put))
    """
    
    dt = T/N  # Divide the options life-span equally by the number of steps taken
    U = np.exp(sigma*np.sqrt(dt)) # CRR equation
    D = 1/U # Using UD = 1
    q_h = (np.exp((r-q)*dt) - D)/(U-D) # risk-neutral probability of "up" movement
    q_l = 1 - q_h # risk-neutral probability of "down" movement
    
    # array for number of "up" moves, converted to float so we can use numpy's "power" method 
    no_of_up_moves = np.arange(N, -1, -1, dtype=float).reshape(-1, 1) 

    # array of asset prices at the end of our time period 
    final_node_values = S_0 * np.power(U, 2 * no_of_up_moves - N)
    asset_prices = final_node_values
    
    if option_type == "C":
        node_values = np.maximum(0, final_node_values - K)
    else: 
        node_values = np.maximum(0, K - final_node_values)
    # The below loop calculates the expected value of the claim from the child nodes, discounted at the risk-free rate, 
    # and then compares that to the payoff from exercising at the parent node, assigning the maximum value as the 
    # effective option price at that node. The empty lists serve as a means of storing the asset prices and effective 
    # option prices at each time-step (iteration), saving these values to variables outside the loop before being 
    # formatted, as they'll be used in the next iteration.
    while len(node_values) != 1:
        node_option_price = []
        asset_price = []
        for i in range(len(node_values) - 1):
            effective_price = np.exp(-r*dt)*(node_values[i] * q_h + node_values[i + 1] * q_l) # risk-neutral expectation
            american_price = asset_prices[i]/U # Utilising S(t-1) = S(t)_u/U
            asset_price.append(american_price)
            if option_type == "C":
                node_option_price.append(max(effective_price, american_price - K))
            else:
                node_option_price.append(max(effective_price, K - american_price))
        asset_prices = np.array(asset_price)
        node_values = np.array(node_option_price)
    
    return f"The corresponding option is valued at £{round(node_values[0][0], 2)}"

Here, I calculated the price of an American call option with a strike price of £20, based on an underlying asset currently priced at £16, with a volatility of 50%, expiring in 2 years. The risk-free rate is 6%, and there is a continuous dividend yield of 5%. As you can see, the theoretical price is approximately £3.05.

In [54]:
american_option_price(16, 20, 0.5, 0.05, r = 0.06, T = 2, option_type = "C")

'The corresponding option is valued at £3.05'

## Cash-or-nothing Binary options
Moving away from vanilla options, I will first price path-independent cash-or-nothing Binary options using the binomial tree approach, before moving on to pricing a wider class of path-dependent exotic options for the rest of the notebook.

In [12]:
def binary_option_price(S_0, K, sigma, q, Q, N = 1000, r = 0, T = 1, option_type = "C"):
    """
    Prices a variety of cash-or-nothing Binary call and put options using a Cox-Ross-Rubinstein valuation
    
    Parameters:
    S_0: Initial asset price
    K: Strike price
    sigma: Asset volatility
    q: Continuous dividend yield
    Q: Fixed payment 
    N: Number of time steps
    r: Interest rate
    T: Number of years
    option_type: Type of option (either call : "C" or put : "P" (although any other input will work for put))
    """
    
    dt = T/N # Divide the options life-span equally by the number of steps taken
    U = np.exp(sigma*np.sqrt(dt)) # CRR equation
    D = 1/U # Using UD = 1
    q_h = (np.exp((r-q)*dt) - D)/(U-D) # risk-neutral probability of "up" movement
    q_l = 1 - q_h # risk-neutral probability of "down" movement
    
    # array for number of "up" moves, converted to float so we can use numpy's "power" method 
    no_of_up_moves = np.arange(N, -1, -1, dtype=float).reshape(-1, 1) 
    
    # array of asset prices at the end of our time period 
    final_node_values = S_0 * np.power(U, 2 * no_of_up_moves - N)

    # risk-neutral probability measure 
    rn_probs = comb(N, no_of_up_moves)  * np.power(q_h, no_of_up_moves) * np.power(q_l, N - no_of_up_moves)
    
    if option_type == "C":
        final_node_payoff = np.where(final_node_values > K, Q, 0) # above threshold
    else:
        final_node_payoff = np.where(final_node_values < K, Q, 0) # below threshold
    
    # Expected payoff w.r.t the risk-neutral measure discounted at the continuously compounded rate
    return f"The corresponding option is valued at £{round((np.exp(-r*T) * sum(rn_probs * final_node_payoff))[0], 2)}"

Here, I calculated the price of a cash-or-nothing put option with a strike price of £100, based on an underlying asset currently priced at £100, with a volatility of 10%, expiring in 10 years, and paying out £100. The risk-free rate is 10%, and there is a continuous dividend yield of 5%. As you can see, the theoretical price is approximately £2.67.

In [55]:
binary_option_price(100, 100, 0.1, 0.05, 100, N = 1000, r = 0.1, T = 10, option_type = "P")

'The corresponding option is valued at £2.67'

# Monte Carlo method
The Monte Carlo method here essentially utilises the central limit theorem, which, put informally, states that the sample mean ($\bar{X}$) follows a normal distribution with mean and variance of the population mean ($\mu$) and the population variance divided by the sample size ($\frac{\sigma^{2}}{N}$), as $N \to \infty$. This method randomly generates a multitude of realisations, effectively constructing a large sample, and uses the consistent sample mean estimator to give an estimate of $\mu$. This sample mean will, again informally, look like a draw from N($\mu$, 0), as $N \to \infty$. 

However, as $\sigma^{2}$ is unknown, we don't know the true variance of the normal distribution draws, requiring an estimate of the standard error from the unbiased and consistent sample variance estimator ($S^{2}$). This means that when constructing confidence intervals, we need to use a t-distribution with N-1 degress of freedom. However, as $N \to \infty$, the t-distribution converges to the standard normal, and since our number of simulations will be rather large, we should expect that upon repeated sampling of the sample mean from this method, the true theoretical option price will be within +/- 1 standard errors from the sample mean in about 68% of the repeated sample mean samples.

## Implementation
Below is a function that prices a large variety of path-dependent options. It will return the theoretical option price estimate, and the 95% confidence interval associated with it. I will assume asset prices follow a log-normal distribution, to be consistent with the Black-Scholes model. 

In [29]:
from scipy import stats

def monte_carlo_option_pricing(S_0, K, sigma, q, H = 0, N = 1000, M = 10000, r = 0, T = 1, option_type = "AA call"):
    """
    Prices a variety of exoic options using a Cox-Ross-Rubinstein valuation
    
    Parameters:
    S_0: Initial asset price
    K: Strike price
    sigma: Asset volatility
    q: Continuous dividend yield
    H: Threshold for Barrier options (default is 0) 
    N: Number of time steps
    r: Interest rate
    T: Number of years
    option_type: Type of option (below is a guideline)
    """
    
    epsilon_i = np.random.normal(size=(N, M)) # N x M array of draws from an N(0, 1) distribution
    dt = T/N # Divide the options life-span equally by the number of steps taken
    miu = (r - q - (sigma ** 2)/2)*dt # mean of the log-normal distribution, without lnS_0
    sd = sigma*np.sqrt(dt) # standard deviation of the log-normal distribution 
    lnS_0 = np.log(S_0) # natural logarithm of the initial price 
    
    # Simulated price process, using the log-normal assumption. Exponentiate everything at end to get the actual price
    prices_array = miu + sd*epsilon_i
    prices_array = np.vstack((np.full((1, M), lnS_0), prices_array))
    prices_array = np.cumsum(prices_array, axis=0) 
    prices_array = np.exp(prices_array)
    
    # Prepare the payoffs from a variety of exotics
    if option_type == "AA call":
        value = np.mean(prices_array, axis=0)
        payoff = np.maximum(0, value - K)
    elif option_type == "AA put":
        value = np.mean(prices_array, axis=0)
        payoff = np.maximum(0, K - value)
    elif option_type == "GA call":
        value = np.power(np.prod(prices_array, axis=0), 1/(N+1))
        payoff = np.maximum(0, value - K)
    elif option_type == "GA put":
        value = np.power(np.prod(prices_array, axis=0), 1/(N+1))
        payoff = np.maximum(0, K - value)
    elif option_type == "OT call":
        value = np.any(prices_array > K, axis = 0) # reaches above a threshold 
        payoff = value
    elif option_type == "OT put":
        value = np.any(prices_array < K, axis = 0) # reaches below a threshold 
        payoff = value
    elif option_type == "UI call":
        condition = np.any(prices_array > H, axis = 0)
        payoff = np.maximum(0, prices_array[-1] - K) * condition
    elif option_type == "UI put":
        condition = np.any(prices_array > H, axis = 0)
        payoff = np.maximum(0, K - prices_array[-1]) * condition
    elif option_type == "UO call":
        condition = np.all(prices_array < H, axis = 0)
        payoff = np.maximum(0, prices_array[-1] - K) * condition
    elif option_type == "UO put":
        condition = np.all(prices_array < H, axis = 0)
        payoff = np.maximum(0, K - prices_array[-1]) * condition
    elif option_type == "DI call":
        condition = np.any(prices_array < H, axis = 0)
        payoff = np.maximum(0, prices_array[-1] - K) * condition
    elif option_type == "DI put":
        condition = np.any(prices_array < H, axis = 0)
        payoff = np.maximum(0, K - prices_array[-1]) * condition
    elif option_type == "DO call":
        condition = np.all(prices_array > H, axis = 0)
        payoff = np.maximum(0, prices_array[-1] - K) * condition
    elif option_type == "DO put":
        condition = np.all(prices_array > H, axis = 0)
        payoff = np.maximum(0, K - prices_array[-1]) * condition
    elif option_type == "FLL call":
        value = np.min(prices_array, axis = 0)
        payoff = np.maximum(0, prices_array[-1] - value) 
    elif option_type == "FLL put":
        value = np.max(prices_array, axis = 0)
        payoff = np.maximum(0, value - prices_array[-1]) 
    elif option_type == "FIL call":
        value = np.max(prices_array, axis = 0)
        payoff = np.maximum(0, value - K) 
    elif option_type == "FIL put":
        value = np.min(prices_array, axis = 0)
        payoff = np.maximum(0, K - value) 
    
        
    sample_mean = np.exp(-r * T) * np.mean(payoff) # Present value the mean payoffs to get an estimate of the option price
    sample_variance = np.var(np.exp(-r * T) * payoff)*M/(M-1) 
    standard_error = np.sqrt(sample_variance/M) # Standard deviation of the sampling distribution 
    t_distribution = stats.t(df=M-1).ppf(0.975)
    upper_bound = sample_mean + standard_error*t_distribution # Upper bound C.I
    lower_bound = sample_mean - standard_error*t_distribution # Lower bound C.I
    
    # Theoretical opion price estimate and 95% C.I assoiciated with it
    return f"The theorertical option price is £{round(sample_mean, 2)}, \
with a 95% C.I of {(round(lower_bound, 2), round(upper_bound, 2))}"

### Guideline for option_type

- "AA" : Arithmetic Asian
- "GA" : Geometric Asian
- "OT" : One-touch
- "UI" : Up-and-in
- "UO" : Up-and-out
- "DI" : Down-and-in
- "DO" : Down-and-out
- "FLL" : Floating lookback
- "FIL" : Fixed lookback

[**NOTE**]: Add "call" or "put" at the end of the above option acronyms to get desired option type.

To assess the validity of the estimate, I calculated the price of a Barrier up-and-out call option with a strike of £110 and a threshold of £130. The underlying is currently priced at £120, with a volatility of 70%, expiring in 1 year, with a risk-free rate of 2%, and a continuous dividend yield of 1%. 

Given the volatility and the rather high strike price, we should anticipate that the price is relatively low since the threshold is likely to be exceeded fairly frequently, and when it's not, the payoff won't be too high. As expected, such an option costs £0.02

In [50]:
print(monte_carlo_option_pricing(120, 110, 0.7, 0.01, H = 130, r = 0.02, T = 1, option_type = "UO call"))

The theorertical option price is £0.02, with a C.I of (0.01, 0.03)


With the methods described above, we can price options with a variety of payoff functions. For example, consider a combination of an up-and-out and down-and-out Barrier call option, where the option expires worthless if a lower or upper threshold is exceeded; otherwise, it has the payout of a normal call option. Another example is an Asian put option in which, instead of taking the geometric or arithmetic means of the prices, we use the harmonic mean. I will now proceed with pricing these types of options to showcase the earlier claim.

In [48]:
def custom_monte_carlo_option_pricing(S_0, K, sigma, q, H_U = 120, H_L = 80, N = 1000, M = 10000, r = 0, T = 1,
                                      option_type = "HA put"):
    """
    Prices a variety of exoic options using a Cox-Ross-Rubinstein valuation
    
    Parameters:
    S_0: Initial asset price
    K: Strike price
    sigma: Asset volatility
    q: Continuous dividend yield
    H: Threshold for Barrier options (default is 0) 
    N: Number of time steps
    r: Interest rate
    T: Number of years
    option_type: Type of option (either HA put or UO/DO call)
    """
    
    epsilon_i = np.random.normal(size=(N, M)) # N x M array of draws from an N(0, 1) distribution
    dt = T/N # Divide the options life-span equally by the number of steps taken
    miu = (r - q - (sigma ** 2)/2)*dt # mean of the log-normal distribution, without lnS_0
    sd = sigma*np.sqrt(dt) # standard deviation of the log-normal distribution 
    lnS_0 = np.log(S_0) # natural logarithm of the initial price 
    
    # Simulated price process, using the log-normal assumption. Exponentiate everything at end to get the actual price
    prices_array = miu + sd*epsilon_i
    prices_array = np.vstack((np.full((1, M), lnS_0), prices_array))
    prices_array = np.cumsum(prices_array, axis=0) 
    prices_array = np.exp(prices_array)
    
    # Prepare the payoffs from a variety of custom options
    if option_type == "HA put":
        value = (N+1)/np.sum(1/prices_array, axis=0)
        payoff = np.maximum(0, K - value)
    else:
        condition_1 = np.all(prices_array < H_U, axis = 0)
        condition_2 = np.all(H_L < prices_array, axis = 0)
        payoff = np.maximum(0, prices_array[-1] - K) * condition_1 * condition_2
        
    sample_mean = np.exp(-r * T) * np.mean(payoff) # Present value the mean payoffs to get an estimate of the option price
    sample_variance = np.var(np.exp(-r * T) * payoff)*M/(M-1) 
    standard_error = np.sqrt(sample_variance/M) # Standard deviation of the sampling distribution 
    t_distribution = stats.t(df=M-1).ppf(0.975)
    upper_bound = sample_mean + standard_error*t_distribution # Upper bound C.I
    lower_bound = sample_mean - standard_error*t_distribution # Lower bound C.I
    
    # Theoretical opion price estimate and 95% C.I assoiciated with it
    return f"The theorertical option price is £{round(sample_mean, 2)}, \
with a 95% C.I of {(round(lower_bound, 2), round(upper_bound, 2))}"

To conclude, I will calculate the prices of the two previously mentioned 'custom' options. I will assume a strike price of £200 for both, with the underlying currently priced at £210. The volatility is 10%, and the options expire in 1.5 years. The risk-free rate is 2%, and there is no continuous dividend yield. For the knock-out combination option, I will consider thresholds of £230 and £180.

As you can see, the theoretical prices are approximately £2.82 and £1.54, respectively.

In [51]:
print(custom_monte_carlo_option_pricing(210, 200, 0.1, 0, r = 0.02, T = 1.5, option_type = "HA put"))
print(custom_monte_carlo_option_pricing(210, 200, 0.1, 0, H_U = 230, H_L = 180, r = 0.02, T = 1.5,
                                        option_type = "UO/DO call"))

The theorertical option price is £2.82, with a C.I of (2.7, 2.93)
The theorertical option price is £1.54, with a C.I of (1.46, 1.62)
